# numpy

## import a module

In [ ]:
import numpy as np

In [ ]:
from numpy import linalg as la

In [ ]:
np2 = __import__('numpy')

In [ ]:
la.__file__

In [ ]:
import sys
print(sys.path)
print(os.environ['PYTHONPATH'])

---

## ufunc
element-wise operations

In [ ]:
a = np.zeros((3, 3))

b = a + 1
print(b)

In [ ]:
a = np.random.rand(3, 3)
print(a < 0.5)

In [ ]:
a = np.random.rand(3, 3)
print((a < 0.2) | (a > 0.8))

### ufunc for high dimension arrays

In [ ]:
a = np.arange(3)
b = np.arange(2)
ab = np.empty((3, 2))
for i in range(3):
    for j in range(2):
        ab[i, j] = a[i] * b[j]
        
ab = a[:, np.newaxis] * b
print(a[:, np.newaxis].shape)
print(ab.shape)
print((a[:, np.newaxis, np.newaxis] * b).shape)

In [ ]:
# between arrays with different dimensions
# np unfun automatically patches enough "np.newaxis" at the leading axes
ab + b

In [ ]:
# dimensions of tailing axes must match
ab + a

In [ ]:
# dimensions of tailing axes must match
ab + a[np.newaxis, :]

In [ ]:
# np.newaxis in the middle
c = np.zeros((3, 4, 2))
c + ab[:, np.newaxis, :]
print(ab[:, np.newaxis, :].shape)

In [ ]:
# ufunc with list, tuple or iterables. Objects are converted to np objects first.
a = np.arange(3)
a + [1, 2, 1]
a + (1, 2, 1)
a + range(3, 6)

### einsum

In [ ]:
ab = a[:, np.newaxis] * b
ab = np.einsum('x,y->xy', a, b)

In [ ]:
c = np.random.rand(3, 4, 2)  # trace the index in the middle and swap the first and last axes
c1 = np.zeros((2, 3))
for i in range(3):
    for j in range(4):
        for k in range(2):
            c1[k, i] = c1[k, i] + c[i, j, k]

c1 = np.einsum('xyz->zx', c)

---

## Fancy index

In [ ]:
# list/nparray works as a sequence of subscripts to index an array
a = np.arange(0, 70, 10)
idx = [1, 3, 5]
print(a[idx])

In [ ]:
# tuple works like (element-wise) index subscripts
a = np.arange(0, 70, 10)
idx = (1, 3, 5)
print(a[idx])

In [ ]:
a = np.zeros((7, 7, 7))
print(a[idx])  # == a[1, 3, 5]

### Common scenarios that fancy indices are used

In [ ]:
# reorder elements of an array
a = np.arange(5)
a[[4, 3, 2, 1, 0]]

In [ ]:
# get sub-block of an array
a = np.arange(125).reshape(5, 5, 5)
a[2:4, 1:4, 1:3]

In [ ]:
# Take elements along axes
idx1 = np.array([1, 3, 2])
idx2 = np.array([0, 2])
idx3 = np.array([1, 3, 4])
result = np.empty([3, 2, 3])
for i, m in enumerate(idx1):
    for j, n in enumerate(idx2):
        for k, o in enumerate(idx3):
            result[i, j, k] = a[m, n, o]
            
print(a[idx1, idx2, idx3])  # Not working as we thought?
print(a[idx1][:, idx2][:, :, idx3])

# To construct high-dimension sub-block of an array
print(a[idx[:, np.newaxis, np.newaxis], idx[:, np.newaxis], idx])

In [ ]:
# Index an array with multiple 1D indices
a = np.arange(216).reshape(6, 6, 6)
print(a[([1, 3, 5], [4, 0, 0])])

print(np.array([a[1, 4],
                a[3, 0],
                a[5, 0]]))

In [ ]:
# Size of the 1D indices must match
print(a[([1, 3, 4], [2, 0])])

In [ ]:
# Almost works like the code below
result = []
for i, j in zip([1, 3, 5], [4, 0, 0]):
    result.append(a[i, j])
print(np.array(result))

In [ ]:
# Avoid to apply multiple indices on non-adjacent axes
a = np.zeros((6, 6, 6, 6))
print(a[:, :, [1, 3, 5], [4, 0, 0]].shape)
print(a[:, [1, 3, 5], :, [4, 0, 0]].shape)
print(a[:, [1, 3, 5], [4, 0, 0], :].shape)

Rule of thumb: If two or more indices are specified to reduce the dimension of a high-dimension array,
first transpose the array and make the target axes adjacent then apply the indices to the adjcent axes.

In [ ]:
# when list/nparray and tuple are both specified, tuple is treated as a list in most cases
a = np.arange(216).reshape(6, 6, 6)
print(a[(1, 3), (2, 0)])
print(a[[1, 3], [2, 0]])

In [ ]:
# Very misleading if tuple and list are mixed in the array subscripts.
a = np.arange(216).reshape(6, 6, 6)
print(a[(1, 3), 2])
print(a[[1, 3], 2])
print(a[[1, 3], [2, 2]])
print(a[[(1, 3), (2, 2)]])

Rule of thumb: Except Tuple[List] which works as exaplained above, we should avoid to mix tuple and lists when indexing an array.


In [ ]:
# What if indices and the base array have different shapes
a = np.arange(0, 70, 10)
idx = np.array([[1, 3, 5]])
print(a[idx])

idx = np.array([[1],
                [3],
                [5]])
print(a[idx])

idx = np.array([[1, 2, 3],
                [3, 4, 5],
                [5, 6, 0]])
print(a[idx])

idx = np.array([1, 3, 5])
print(a[idx[:, np.newaxis, np.newaxis]].shape)

Rule of thumb: The shape of the return array with fancy index is not determined by the shape of the base (original) array. The shape of the return array is determined by the shape of the index list/array

### Mask array

In [ ]:
a = np.arange(0, 70, 10)
mask = np.array([True, False, True, False, False, False, True])
a[mask]

In [ ]:
# The return (masked) array is a one-dimension array, regardless of the shape of the original array.
a = np.random.rand(3, 3, 3)
mask = (a < 0.2) | (a > 0.8)
print(mask)
print(a[mask])

# The dimension of mask must be less than the dimension of the array.
mask = np.random.rand(3, 3) < 0.5
print(a[mask].shape)

In [ ]:
# conversion between indices and mask array
mask = np.random.rand(3, 4) < 0.5
idx0, idx1 = np.where(mask)

a = np.zeros((4, 4, 4))
mask = np.zeros(a.shape, dtype=bool)
mask[idx0, idx1] = True

In [ ]:
# Mask array to filter nan
np.isnan(a)
np.isfinite(a)

### assignment with fancy indices

In [ ]:
a = np.zeros((3, 3))
a[1:3, 2] = 1
print(a)

In [ ]:
a = np.zeros((3, 3))
a[[1, 2], 2] = 1
print(a)

In [ ]:
a = np.zeros((3, 3))
a[[0, 2], 2] = 1
print(a)

In [ ]:
a = np.zeros((3, 3))
a[[0, 2], [0, 2]] = 1
print(a)

In [ ]:
# assignment for a subblock
a = np.zeros((3, 3))
idx = np.array([0, 2])
a[idx][:, idx] = 1
print(a)
a[idx[:, np.newaxis], idx] = 1
print(a)

---

## dtype

In [ ]:
print(np.typeDict)
print(np.typecodes)

# One np object can have only one dtype
a = np.array([2, 0, 9])
print(a.dtype)
a = np.array([2, 0, 2.5])
print(a.dtype)
a = np.array([2, 0, 'a'])
print(a.dtype)
a = np.array([2, 0, ['a']])
print(a.dtype)

# float64 and int64 by default for most array creation function
a = np.zeros(3)
print(a.dtype)
a = np.arange(3)
print(a.dtype)
a = np.array([2, 3, 5.5])
print(a.dtype)

### The evil of nan

In [ ]:
# comparison
print(np.inf == np.inf)
print(np.nan == np.nan)

In [ ]:
# Use isnan to compare a number with nan
a = np.random.rand(5)
a[2] = np.nan
np.isnan(a)

In [ ]:
# When casting nan to integer, bool, str, datetime?
nan = np.array(np.nan)
print(nan.astype(int))
print(nan.astype(np.bool))
print(nan.astype(np.datetime64))
print(nan.astype(np.dtype('U4')))  # a datatype of 4-byte string
print(nan.astype(np.object))

In [ ]:
# What if assign nan to a variable of integer / bool / str / datetime type?
a = np.zeros(1, dtype=int)
a[0] = np.nan
a = np.zeros(1, dtype=bool)
a[0] = np.nan
a = np.zeros(1, dtype='U4')
a[0] = np.nan

np functions to manipulate arrays with nans
* np.min -> np.nanmin
* np.max -> np.nanmax
* np.sum -> np.nansum
* np.mean -> np.nanmean
* np.std -> np.nanstd

In [ ]:
# Powerful np.dtype, works like pandas dataframe
np.dtype([('name', 'U16'), ('grades', np.int)])

---

## nparray data structure

Attributes of an array that presents the array data structure
* dimensions
* shape
* itemsize
* strides
* data buffer
* offset wrt the beginning of the data buffer

In [ ]:
a = np.random.rand(3, 2, 4)
print(a.data, a.base is None)
print(a.itemsize)
print(a.ndim)
print(a.shape)
print(a.strides)

In [ ]:
# subarray
b = a[1:, 0, 2]
print(b.data, b.base is a)
print(b.itemsize)
print(b.ndim)
print(b.shape)
print(b.strides)

b = a[[1, 2]]
print(b.data, b.base is a)
print(b.itemsize)
print(b.ndim)
print(b.shape)
print(b.strides)

In [ ]:
# Transpose does not change the data in memory
b = a.transpose(2, 1, 0)
print(b.data, b.base is a)
print(b.strides)

"view" vs copy
* An array-view does not allocate new memory to hold data. It reuses the data buffer of original array with different array structure (strides, shape, ndim, etc).
* A numpy function returns the view of an array if the return array can be represented by the data buffer of the base (original) array with proper values of array structure.
* How to check
* The attribute `.base` of array view is not None
* The attribute `.flags.owndata` is False

In [ ]:
# sliced array is view
a = np.random.rand(3, 2, 4)
b = a[:2, 1:, 3:]
print(a.base is None)
print(a.flags.owndata)
print(b.base is not None, b.base is a)
print(b.flags.owndata)

# Functions swapaxes, transpose, diagonal returns an array view
np.swapaxes(a, 0, -1).base
np.rollaxis(a, 2, 0).base
a.transpose(1, 2, 0).base
a[0].diagonal().base
a.T.base

# ravel, reshape returns an array view if possible
a.reshape(12, 2).base
a.ravel().base
a.transpose(1, 2, 0).reshape(12, 2).base
a.transpose(1, 2, 0).ravel().base
a.transpose(2, 1, 0).reshape(12, 2).base

# Both fancy indices and array mask return copy of an array
a[[1]].base
a[np.array([False, False, True])]

In [ ]:
# When editting an array
a = np.zeros((3, 2, 4))
b = a[:, 1, :]
b[1] = 1
print(a)

In [ ]:
# Implicit array view: inplace operations
a = np.zeros((3, 2, 4))
a = a + 1
a += 1

In [ ]:
# One can select a sub-array to simplify the assignment statement
a = np.zeros((5, 5, 5))
a[1, 2, [0, 3, 4]] = 1
print(a)

sub_array_at_1_2 = a[1, 2]
sub_array_at_1_2[[0, 3, 4]] = 2
print(a)

In [ ]:
# performance differences
%timeit a = a + 1
%timeit a += 1

Rule of thumb: When a function returns an array that might be a view of input array, call .copy(). Whenever you unsure, call .copy().

In [ ]:
# Question: what would happen if we apply inplace operations to the view of the array itself
a = np.random.rand(5, 5, 5)
b = a.transpose(2, 0, 1)
b += a

# This is a long-live bug (https://github.com/numpy/numpy/issues/1683) until recent numpy release (1.12)

---

## np array vs list
### subarray

In [ ]:
l = list(range(7))
a = np.arange(7)
print(l[::3])
print(a[::3])

### ufunc

In [ ]:
l = [1, 2, 3]
a = np.arange([1, 2, 3])
print(l == l)
print(a == l)
print(np.all(a == l))
print(np.array_equal(a, l))

### append, sort, index

In [ ]:
lst.append(x)
arr = np.append(arr, x)
arr = np.hstack([arr, x])

lst.sort()  # sort inplace
arr.sort()  # sort inplace

sorted(lst)  # no side-effects
np.sort(arr)  # no side-effects

lst.index(x)
np.where(arr == x)[0][0]

### memory view and implicit memory cost

In [ ]:
a = [0, 1, 2, 3, 4, 5]
b = a[2:4]
b[0] = 10
print(a)
a = None

a = np.array([0, 1, 2, 3, 4, 5])
b = a[2:4]
b[0] = 10
print(a)
a = None

---

## Vector-oriented programming

In [ ]:
# Given a set of points, find the largest 3 distances between points if either point is inside the circle with raidus r0
def py_max_distances(coordinates, r0):
    max_dists = [0, 0, 0]
    for i, r1 in enumerate(coordinates):
        for j, r2 in enumerate(coordinates):
            if i > j and ((np.linalg.norm(r1) < r0) ^ (np.linalg.norm(r2) < r0)):
                dist = np.linalg.norm(r1 - r2)
                if dist > max_dists[0]:
                    max_dists = sorted(max_dists[1:] + [dist])
    return max_dists

def np_max_distances(coordinates, r0):
    pair_dists = np.linalg.norm(coordinates[:, np.newaxis, :] - coordinates, axis=2)
    points_in_r0 = np.linalg.norm(coordinates, axis=1) < r0
    mask_r0 = points_in_r0[:, np.newaxis] ^ points_in_r0
    mask_uniq_dists = np.tril(np.ones_like(mask_r0), k=-1)
    uniq_dists = pair_dists[mask_r0 & mask_uniq_dists]
    return np.partition(uniq_dists, -3)[-3:]

In [ ]:
coordinates = np.random.rand(10, 2)
print(py_max_distances(coordinates, 0.8))
print(np_max_distances(coordinates, 0.8))

In [ ]:
coordinates = np.random.rand(500, 2)
%time py_max_distances(coordinates, 0.8)
%time np_max_distances(coordinates, 0.8)

---

## numpy array serialization

### 1. pickle
* Compatibility between Python versions
* Insecure

### 2. np.save and np.load

In [ ]:
np.save('/tmp/example.npy', a)
a = np.load('/tmp/example.npy')

### 3. to python builtin data object

In [ ]:
a = np.arange(3)
a.tolist()

### 4. readable data on disk

In [ ]:
a.tofile('/tmp/example.txt')
np.fromfile('/tmp/example.txt')

### 5. 3rd party library: h5py, pytables, ...

In [ ]:
import h5py
with h5py.File('example.h5') as h5f:
    h5f['data/group/to/store'] = np.arange(3)

### 6. json
* good readability
* cross-platform compatibility

In [ ]:
import json
a = np.arange(3)
json.dumps(a)

# solution 1
json.dumps(a.tolist())
# solutiohn 2
import pandas as pd
pd.DataFrame(a).to_json()